# Задание 2. Проверить, насколько хорошо работает NER

Данные брать из http://www.labinform.ru/pub/named_entities/

1. проверить NER из nltk/spacy/deeppavlov.

2. написать свой NER, попробовать разные подходы.

 - передаём в сетку токен и его соседей.

 - передаём в сетку только токен.
 - свой вариант.

3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

In [1]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python -m spacy download en_core_web_md

2022-11-08 16:51:56.084085: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 3.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
import corus
from corus import load_ne5

import re
import nltk

nltk.download('ieer')

import spacy
from spacy import displacy
import en_core_web_md

[nltk_data] Downloading package ieer to /root/nltk_data...
[nltk_data]   Package ieer is already up-to-date!


In [4]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-11-08 16:52:19--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.22MB/s    in 0.8s    

2022-11-08 16:52:21 (2.22 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [5]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
!rm collection5.zip

In [7]:
!ls

Collection5  sample_data


In [8]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='492',
    text='Д.Медведев уволил ряд высокопоставленных военных\r\n\r\nПрезидент РФ Дмитрий Медведев в пятницу, 23 декабря, освободил от должности ряд высокопоставленных военных.\r\n\r\nУказом главы государства от должности заместителя командующего Черноморским флотом был освобожден вице-адмирал Сергей Меняйло. Также Д.Медведев уволил с военной службы генерал-майора Багира Фатуллаева.\r\n\r\nКроме того, глава государства освободил от занимаемой должности полковника медицинской службы Алексея Никитина, начальника 2-го управления - заместителя начальника главного военно-медицинского управления Министерства обороны РФ; полковника Николая Ярцева, начальника филиала военного учебно-научного центра Военно-воздушных сил "Военно-воздушная академия им.Н.Жуковского и Ю.Гагарина" (Сызрань, Самарская область).\r\n\r\nПрезидент РФ назначил полковника Дмитрия Горбатенко начальником управления подготовки войск Западного военного округа.\r\n\r\nНапомним, что летом этого года Д.Медв

In [9]:
import nltk

In [10]:
document = next(records).text
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nltk.pos_tag(nltk.word_tokenize(document))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


[('День', 'JJ'),
 ('памяти', 'NNP'),
 ('и', 'NNP'),
 ('скорби', 'NNP'),
 ('начался', 'NNP'),
 ('в', 'NNP'),
 ('Брянске', 'NNP'),
 ('с', 'NNP'),
 ('традиционного', 'NNP'),
 ('ночного', 'NNP'),
 ('митинга-реквиема', 'JJ'),
 ('Традиционная', 'NNP'),
 ('молодежная', 'NNP'),
 ('патриотическая', 'NNP'),
 ('акция', 'NNP'),
 ('``', '``'),
 ('Зажги', 'FW'),
 ('свечу', 'NN'),
 ("''", "''"),
 ('состоялась', 'CC'),
 ('в', 'NNP'),
 ('Брянске', 'NNP'),
 ('на', 'NNP'),
 ('Партизанской', 'NNP'),
 ('Поляне', 'NNP'),
 ('в', 'NNP'),
 ('ознаменование', 'NNP'),
 ('72-й', 'JJ'),
 ('годовщины', 'NNP'),
 ('начала', 'NNP'),
 ('Великой', 'NNP'),
 ('Отечественной', 'NNP'),
 ('войны', 'NNP'),
 ('в', 'NNP'),
 ('ночь', 'NNP'),
 ('на', 'VBD'),
 ('22', 'CD'),
 ('июня', 'NN'),
 ('.', '.'),
 ('Ночной', 'CC'),
 ('митинг', 'JJ'),
 ('на', 'NNP'),
 ('Партизанской', 'NNP'),
 ('поляне', 'NNP'),
 ('в', 'NNP'),
 ('День', 'NNP'),
 ('памяти', 'NNP'),
 ('и', 'NNP'),
 ('скорби', 'NNP'),
 ('проводится', 'NNP'),
 ('в', 'NNP'),
 ('го

In [11]:
[(' '.join(c[0] for c in chunk), chunk.label()) 
            for chunk in nltk.ne_chunk(
                nltk.pos_tag(nltk.word_tokenize(document))
                ) if hasattr(chunk, 'label')
]

[('День', 'PERSON'),
 ('Брянске', 'PERSON'),
 ('Брянске', 'PERSON'),
 ('Поляне', 'PERSON'),
 ('Великой Отечественной', 'PERSON'),
 ('Памяти', 'PERSON'),
 ('Великой Отечественной', 'PERSON'),
 ('Стены Скорби', 'PERSON'),
 ('Великой Отечественной', 'PERSON'),
 ('Белобережская', 'ORGANIZATION'),
 ('Великой Отечественной', 'PERSON'),
 ('Василий Бударин', 'PERSON'),
 ('Яков Рожнов', 'PERSON'),
 ('Великой Отечественной', 'PERSON'),
 ('Вечному', 'PERSON'),
 ('Снежеть', 'PERSON')]

In [12]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from razdel import tokenize

words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):       
        result = 'None'        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
        words.append([token.text, result])        
    words_docs.extend(words)

In [14]:
import pandas as pd

df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

None        218808
PER          21167
ORG          13621
LOC           4553
GEOPOLIT      4352
MEDIA         2480
Name: tag, dtype: int64

In [15]:
df_words.head()

,word,tag
0,В,PER
1,.,PER
2,Путин,PER
3,переназначил,None
4,одного,None


In [16]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, Conv2D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [17]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [18]:
train_x.apply(len).max(axis=0)

55

In [19]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [20]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
def custom_standardization(input_data):
    return input_data

def data_preparation(train_data, seq_len=1, vocab_size = 30000):    

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [22]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [23]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 1, vocab_size=vocab_size)

mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12421/12421 [==============================] - 410s 33ms/step - loss: 0.2833 - accuracy: 0.9180 - val_loss: 0.2042 - val_accuracy: 0.9403
Epoch 2/5
 4879/12421 [==========>...................] - ETA: 3:48 - loss: 0.1267 - accuracy: 0.9629

KeyboardInterrupt: ignored

In [ ]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 5, vocab_size=vocab_size)

mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

In [ ]:
vectorize_layer = data_preparation(train_data, seq_len = 10, vocab_size=vocab_size)

mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Deeppavlov не хочет устанавливаться(((

Пробовала Spacy - похожая история.